## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,2021-01-13 04:24:26,62.4560,-114.3525,8.60,85,75,8.05,light snow
1,1,Sao Joao Da Barra,BR,2021-01-13 04:22:33,-21.6403,-41.0511,76.08,86,94,14.38,overcast clouds
2,2,Moses Lake,US,2021-01-13 04:24:27,47.1301,-119.2781,42.80,87,90,16.46,overcast clouds
3,3,Cedar City,US,2021-01-13 04:24:27,37.6775,-113.0619,26.60,54,1,4.61,clear sky
4,4,Carbonia,IT,2021-01-13 04:24:27,39.1672,8.5222,53.60,87,75,20.71,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Sao Joao Da Barra,BR,2021-01-13 04:22:33,-21.6403,-41.0511,76.08,86,94,14.38,overcast clouds
6,6,Kapaa,US,2021-01-13 04:24:28,22.0752,-159.3190,75.20,73,1,6.91,clear sky
9,9,Rikitea,PF,2021-01-13 04:24:29,-23.1203,-134.9692,78.10,77,2,17.58,clear sky
12,12,Souillac,MU,2021-01-13 04:24:29,-20.5167,57.5167,78.80,83,75,11.50,broken clouds
14,14,Kralendijk,BQ,2021-01-13 04:24:30,12.1500,-68.2667,77.00,88,20,10.36,few clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                243
City                   243
Country                241
Date                   243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sao Joao Da Barra,BR,76.08,overcast clouds,-21.6403,-41.0511,
6,Kapaa,US,75.20,clear sky,22.0752,-159.3190,
9,Rikitea,PF,78.10,clear sky,-23.1203,-134.9692,
12,Souillac,MU,78.80,broken clouds,-20.5167,57.5167,
14,Kralendijk,BQ,77.00,few clouds,12.1500,-68.2667,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sao Joao Da Barra,BR,76.08,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
6,Kapaa,US,75.20,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
9,Rikitea,PF,78.10,clear sky,-23.1203,-134.9692,Pension Maro'i
12,Souillac,MU,78.80,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
14,Kralendijk,BQ,77.00,few clouds,12.1500,-68.2667,Divi Flamingo Beach Resort & Casino


In [19]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Desc</dt<>dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))